In [1]:
import os
os.chdir('../../')

In [2]:
import numpy as np
import pandas as pd

from scr.util import *
from scr.feature_engineering.engineering import *

In [3]:
df_train = pd.read_csv('data/null_survey/train_all_ok.csv')
df_test = pd.read_csv('data/null_survey/test_all_ok.csv')

In [9]:
# 特徴量作成前のデータセット

# display(df_train.head())
# display(df_test.head())

,id,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,...,NumberOfTrips,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,ProdTaken,Marry,Car,Child,AgeGroup
0,0,50,Self Enquiry,2,6.803505,Large Business,male,1.0,4.0,Basic,...,5.0,1,4,Executive,12.444719,1,Single,No Car,0_child,50s
1,1,56,Company Invited,1,6.734592,Salaried,male,1.0,4.0,Standard,...,2.0,1,4,Senior Manager,12.910348,0,Divorced,Has Car,0_child,50s
2,2,35,Self Enquiry,1,6.398595,Large Business,female,1.0,3.0,Basic,...,4.0,0,4,Executive,12.535901,1,Married,No Car,0_child,30s
3,3,37,Self Enquiry,2,6.985642,Small Business,female,1.0,3.0,Standard,...,1.0,0,5,Senior Manager,12.697122,0,Divorced,Has Car,0_child,30s
4,4,48,Company Invited,3,6.928538,Small Business,female,1.0,3.0,Basic,...,4.0,0,4,Executive,12.462403,1,Single,Has Car,0_child,40s


,id,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,Marry,Car,Child,AgeGroup
0,3489,48,Self Enquiry,2,6.660575,Small Business,male,1.0,4.0,Super Deluxe,3.0,7.0,0,3,AVP,13.116247,Married,Has Car,0_child,40s
1,3490,30,Self Enquiry,2,6.580639,Small Business,female,1.0,4.0,Standard,3.0,4.0,1,3,Senior Manager,12.611541,Married,No Car,0_child,30s
2,3491,25,Self Enquiry,1,6.293419,Salaried,female,1.0,4.0,Basic,3.0,1.0,0,3,Executive,12.468441,Divorced,No Car,0_child,20s
3,3492,21,Company Invited,2,6.042633,Salaried,male,1.0,4.0,Basic,4.0,1.0,0,3,Senior Manager,12.467960,Divorced,Has Car,0_child,20s
4,3493,41,Company Invited,1,6.042633,Salaried,male,1.0,4.0,Basic,3.0,1.0,0,4,Executive,12.501838,Single,Has Car,0_child,40s


In [4]:
# NumberOfTrips を数値変換
# 特徴量作成用

df_train['NumberOfTrips_log'] = np.log1p(df_train['NumberOfTrips'])
df_test['NumberOfTrips_log'] = np.log1p(df_test['NumberOfTrips'])

# 特徴量作成

## TypeOfContactNuLL
TypeOfContact が欠損値の場合を、明らかに学習させるための特徴量

In [5]:
df_train['TypeofContactNULL'] = df_train['TypeofContact'].apply(make_TypeOfContactNULL)
df_test['TypeofContactNULL'] = df_test['TypeofContact'].apply(make_TypeOfContactNULL)

## Motivation
顧客の旅行に対する意欲を表す特徴量

In [6]:
df_train['Motivation'] = df_train['NumberOfPersonVisiting'] * df_train['NumberOfFollowups'] + (df_train['Passport'] * 10)
df_test['Motivation'] = df_test['NumberOfPersonVisiting'] * df_test['NumberOfFollowups'] + (df_test['Passport'] * 10)

## EconomicPower
顧客の経済力を表す特徴量

In [7]:
df_train['EconomicPower'] = df_train['MonthlyIncome'] * (4 - df_train['CityTier']) * df_train['Gender'].apply(make_motivation_gender)
df_test['EconomicPower'] = df_test['MonthlyIncome'] * (4 - df_test['CityTier']) * df_test['Gender'].apply(make_motivation_gender)

## Child01
顧客に子どもがいるかどうかを表す特徴量

In [8]:
df_train['Child01'] = df_train['Child'].apply(make_child01)
df_test['Child01'] = df_test['Child'].apply(make_child01)

## TripEasier
旅行の行きやすさを表す特徴量

In [9]:
df_train['TripEasier'] = (5 - df_train['NumberOfPersonVisiting']) * df_train['ProductPitched'].apply(make_TripEasier_ProductPitched) * df_train['Marry'].apply(make_TripEasier_Marry) * df_train['Child'].apply(make_TripEasier_Child)
df_test['TripEasier'] = (5 - df_test['NumberOfPersonVisiting']) * df_test['ProductPitched'].apply(make_TripEasier_ProductPitched) * df_test['Marry'].apply(make_TripEasier_Marry) * df_test['Child'].apply(make_TripEasier_Child)

## SalesPerformance
営業担当者の単位時間当たりのパフォーマンス

In [10]:
df_train['SalesPerformance'] = df_train['NumberOfFollowups'] * df_train['PitchSatisfactionScore'] / df_train['DurationOfPitch']
df_test['SalesPerformance'] = df_train['NumberOfFollowups'] * df_test['PitchSatisfactionScore'] / df_test['DurationOfPitch']

## LivingCost
生活コストを表す数値

In [11]:
df_train['LivingCost'] = df_train['MonthlyIncome'] / (df_train['CityTier'].apply(make_LivingCost_CityTier) * (1 + df_train['Child'].apply(make_LivingCost_Child) + df_train['Marry'].apply(make_LivingCost_Marry) + df_train['Car'].apply(make_LivingCost_Car)))
df_test['LivingCost'] = df_test['MonthlyIncome'] / (df_test['CityTier'].apply(make_LivingCost_CityTier) * (1 + df_test['Child'].apply(make_LivingCost_Child) + df_test['Marry'].apply(make_LivingCost_Marry) + df_test['Car'].apply(make_LivingCost_Car)))

## EconomicStability
経済安定性指標

In [12]:
df_train['EconomicStability'] = df_train['MonthlyIncome'] * df_train['Occupation'].apply(make_EconomicStability_Occupation) * df_train['Designation'].apply(make_EconomicStability_Disignation)
df_test['EconomicStability'] = df_test['MonthlyIncome'] * df_test['Occupation'].apply(make_EconomicStability_Occupation) * df_test['Designation'].apply(make_EconomicStability_Disignation)

## TripFreaqency
旅行頻度

In [13]:
df_train['TripFreaqency'] = df_train['NumberOfTrips'].apply(make_TripFreaqency)
df_test['TripFreaqency'] = df_test['NumberOfTrips'].apply(make_TripFreaqency)

## TravelCost
総旅行費の推定

In [14]:
df_train['TravelCost'] = df_train['PreferredPropertyStar'].apply(make_TravelCost_PreferredPropertyStar) * (1 + df_train['NumberOfTrips_log'])
df_test['TravelCost'] = df_test['PreferredPropertyStar'].apply(make_TravelCost_PreferredPropertyStar) * (1 + df_test['NumberOfTrips_log'])

## EconomicSegment
経済力のセグメント

In [15]:
df_train['EconomicSegment'] = df_train['Occupation'] + df_train['Designation'] + df_train['CityTier'].astype(str)
df_test['EconomicSegment'] = df_test['Occupation'] + df_test['Designation'] + df_test['CityTier'].astype(str)

df_train = mapping_EconomicSegment(df_train)
df_test = mapping_EconomicSegment(df_test)

## PacakgeMatch
営業担当者がセールスした商品と、顧客の希望するホテルのランクがマッチしているかを表す数値

In [16]:
product_to_star = {
    'Basic': 3,
    'Standard': 3,
    'Deluxe': 4,
    'Super Deluxe': 5,
    'King': 5
}

# 製品と希望ホテルの星評価の適合度を計算
def calculate_fit(product, preferred_star):
    ideal_star = product_to_star[product]
    max_star_difference = 5  # 星評価の最大差（1星から5星）
    return max(0, 1 - abs(ideal_star - preferred_star) / max_star_difference)

# 新しい特徴量をデータフレームに追加
df_train['PackageMatch'] = df_train.apply(lambda row: calculate_fit(row['ProductPitched'], row['PreferredPropertyStar']), axis=1)
df_test['PackageMatch'] = df_test.apply(lambda row: calculate_fit(row['ProductPitched'], row['PreferredPropertyStar']), axis=1)

In [17]:
df_train[['PackageMatch', 'ProductPitched', 'PreferredPropertyStar']]

,PackageMatch,ProductPitched,PreferredPropertyStar
0,1.0,Basic,3.0
1,1.0,Standard,3.0
2,1.0,Basic,3.0
3,0.8,Standard,4.0
4,0.8,Basic,4.0
...,...,...,...
3484,1.0,Basic,3.0
3485,0.6,Basic,5.0
3486,1.0,Standard,3.0
3487,0.6,King,3.0


# 特徴量のデータ型

In [18]:
feature = [
    'Age',
    'TypeofContact',
    'CityTier',
    'DurationOfPitch',
    'Occupation',
    'Gender',
    'NumberOfPersonVisiting',
    'NumberOfFollowups',
    'ProductPitched',
    'PreferredPropertyStar',
    'NumberOfTrips',
    'Passport',
    'PitchSatisfactionScore',
    'Designation',
    'MonthlyIncome',
    'Marry',
    'Car',
    'Child',
    # 以下、作成特徴量
    'AgeGroup',
    'TypeofContactNULL',
    'Motivation',
    'EconomicPower',
    'Child01',
    'TripEasier',
    'SalesPerformance',
    'LivingCost',
    'EconomicStability',
    'TripFreaqency',
    'TravelCost',
    'NumberOfTrips_log',
    'EconomicSegment',
    'PackageMatch'
]

In [19]:
float_columns = ['DurationOfPitch', 'MonthlyIncome', 'EconomicPower', 'SalesPerformance',
                'LivingCost', 'EconomicStability', 'TravelCost', 'NumberOfTrips_log',
                'PackageMatch']
int_columns = ['Age', 'CityTier', 'NumberOfPersonVisiting', 'NumberOfFollowups', 'PreferredPropertyStar',
                'NumberOfTrips', 'Passport', 'PitchSatisfactionScore', 'TypeofContactNULL',
                'Motivation', 'Child01', 'TripEasier', 'TripFreaqency', 'EconomicSegment'
                ]

In [20]:
for col in float_columns:
    df_train[col] = df_train[col].astype(float)
    df_test[col] = df_test[col].astype(float)

for col in int_columns:
    df_train[col] = df_train[col].astype(int)
    df_test[col] = df_test[col].astype(int)

# CSV出力

In [21]:
if 'test' in df_train.columns:
    df_train = df_train.drop(columns='test', axis=1)

In [25]:
df_train.to_csv('data/feature_engineered/null_cat_cat/train_null_cat_cat.csv', index=False)
df_test.to_csv('data/feature_engineered/null_cat_cat/test_null_cat_cat.csv', index=False)